In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import statsmodels.api as sm
from scipy.stats import linregress

import cartopy.crs as ccrs
import cartopy.feature as cfeature
from matplotlib.patches import Wedge
from matplotlib.colors import Normalize
from matplotlib.cm import ScalarMappable, Reds

plt.rcParams['font.family'] = 'Arial'
plt.rcParams['axes.unicode_minus'] = False  


xls_new = pd.ExcelFile(r'NDVI_all.xlsx')


dfs_new = [xls_new.parse(x) for x in xls_new.sheet_names]



def check_recovery(row):
    recovery_status = 2
    year_value = None
      
    
    for i in range(len(row)):
        if (row[i:i+2] > 0).all():
            recovery_status = 1
            year_value = i+1
            break

    if recovery_status == 2:
        if (row[i:i+4] < 0).all() and (row[i:i+4].diff().sum()  <= 0):
            recovery_status = 3
            year_value = 999
        if recovery_status == 2:
            year_value = 999

    return recovery_status, year_value



for i, df in enumerate(dfs_new[:2], start=1):
    lon=df['lon']
    lat=df['lat']
    landuse=df['Landuse']

    if i==1:
        df = df.iloc[:, -8:].sub(df.iloc[:, -9], axis=0)
    else:
        df = df.iloc[:, -8:].sub(df.iloc[:, -9], axis=0) 
    

    df[['Recovery', 'Year']] = df.apply(check_recovery, axis=1, result_type='expand').astype(int)
    df['Landuse']=landuse
    df['lon']=lon
    df['lat']=lat

    df=df[df['Recovery']==1]
    
    print('Mean',df['Year'].mean())
    print('Median',df['Year'].median())
    print(len(df))
    print('ENF',len(df[df['Landuse']==4]))
    
    

    

    def get_color(year):
        if year == 1:
            return 'darkgreen'
        elif year == 2:
            return 'green'
        elif year == 3:
            return 'limegreen'
        elif year == 4:
            return 'yellowgreen'
        elif year == 5:
            return '#FFBDD0'
        elif year == 6:
            return '#FF859A'
        elif year == 7:
            return '#F94713'
        elif year == 999:
            return 'red'
        else:
            return 'black'

    fig = plt.figure(figsize=(5, 5), dpi=600)
    ax = plt.axes(projection=ccrs.PlateCarree())
    ax.set_global()


    ax.add_feature(cfeature.COASTLINE)
    ax.add_feature(cfeature.BORDERS, linestyle=':')
    ax.add_feature(cfeature.LAND)


    for index, row in df.iterrows():
        lon = row['lon']
        lat = row['lat']
        year = row['Year']

        color = get_color(year)

        ax.scatter(lon, lat, color=color, s=0.6, marker='o', transform=ccrs.PlateCarree()) 
        
        

    count_1 = df[df['Year'] == 1].shape[0]
    count_2 = df[df['Year'] == 2].shape[0]
    count_3 = df[df['Year'] == 3].shape[0]
    count_4 = df[df['Year'] == 4].shape[0]
    count_5 = df[df['Year'] == 5].shape[0]
    count_6 = df[df['Year'] == 6].shape[0]


    year_labels = ['1', '2', '3', '4', '5', '6']
    year_values = [count_1, count_2, count_3, count_4, count_5, count_6]
    

    inset_ax = fig.add_axes([0.14, 0.306, 0.13, 0.2])  # [left, bottom, width, height]
    inset_ax.bar(year_labels, year_values, color=['darkgreen', 'green', 'limegreen', 'yellowgreen', '#FFBDD0', '#FF859A', '#F94713', 'red'])

    inset_ax.tick_params(axis='x', labelsize=6, rotation=30, pad=0.1)
    inset_ax.tick_params(axis='y', labelsize=6, rotation=90, pad=0.1)


    inset_ax.xaxis.set_label_position('top')
    inset_ax.xaxis.tick_top()
    inset_ax.yaxis.tick_right()
    
    inset_ax.spines['left'].set_visible(False)
    inset_ax.spines['bottom'].set_visible(False)
    

    pie_ax = fig.add_axes([0.62, 0.31, 0.14, 0.14])  # [left, bottom, width, height]
    wedges, texts, autotexts = pie_ax.pie(year_values, colors=['darkgreen', 'green', 'limegreen', 'yellowgreen', '#FFBDD0', '#FF859A', '#F94713', 'red'], autopct='%1.1f%%', startangle=90, pctdistance=0.9)

    for a, autotext in enumerate(autotexts):
        autotext.set_fontsize(6)
        x, y = autotext.get_position()
        angle = wedges[a].theta2 - (wedges[a].theta2 - wedges[a].theta1) / 2
        x = 1.3 * x
        y = 1.3 * y
        
        if a==0:
            y =1.2*y
            x =1.5*x

        autotext.set_position((x, y))
    
#     plt.savefig(f"Recovery_Years_NDVI_{i}.jpg",bbox_inches = 'tight',dpi=600)
    
#     plt.show()
#     break